<a href="https://colab.research.google.com/github/rwagler/Exercise-Digital-Business-and-Platfroms/blob/main/exercise_07_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the KumoRFM Python SDK
KumoRFM provides an SDK in Python. The Kumo SDK is available for Python 3.9 to Python 3.13

In [ ]:
!pip install kumoai

In [ ]:
import kumoai.experimental.rfm as rfm

You will need an API key to make calls to KumoRFM.
Use the widget below to generate one for free by clicking "Generate API Key".
If you don't have a KumoRFM account, the widget will prompt you to signup.

You will see the following when your key has been created successfully:

<div align="left">
  <img src="https://kumo-sdk-public.s3.us-west-2.amazonaws.com/rfm-colabs/api-key-created.png" width="300" />
</div>

In [ ]:
import os

from google.colab import userdata
userdata.get('KUMO_API_KEY')

if not os.environ.get("KUMO_API_KEY"):
    rfm.authenticate()

In [ ]:
# Initialize a Kumo client with your API key:
KUMO_API_KEY = os.environ.get("KUMO_API_KEY")
rfm.init(api_key=KUMO_API_KEY)

## Load the Data

In [ ]:
root = 's3://kumo-sdk-public/rfm-datasets/ecom'
path_dict = {
    'users': f'{root}/users.parquet',
    'items': f'{root}/items.parquet',
    'views': f'{root}/views.parquet',
    'orders': f'{root}/orders.parquet',
    'returns': f'{root}/returns.parquet',
}

In [ ]:
# load data into pandas dataframes
import pandas as pd

df_users = pd.read_parquet(path_dict['users'])
df_items = pd.read_parquet(path_dict['items'])
df_views = pd.read_parquet(path_dict['views'])
df_orders = pd.read_parquet(path_dict['orders'])
df_returns = pd.read_parquet(path_dict['returns'])

### Familiarize with the data structure

#### User Data

In [ ]:
df_users.head(5)

In [ ]:
df_users.dtypes

In [ ]:
df_users.info()

#### Items

In [ ]:
df_items.head(5)

In [ ]:
df_items.dtypes

In [ ]:
df_items.info()

#### Orders

In [ ]:
df_orders.head(5)

In [ ]:
df_orders.dtypes

In [ ]:
df_orders.info()

#### Returns

In [ ]:
df_returns.head(5)

In [ ]:
df_returns.dtypes

In [ ]:
df_returns.info()

#### Views

In [ ]:
df_views.head(5)

In [ ]:
df_views.dtypes

In [ ]:
df_views.info()

## Turn the data into KumoRFM tables

In [ ]:
users = rfm.LocalTable(df_users, name="users").infer_metadata()
orders = rfm.LocalTable(df_orders, name="orders").infer_metadata()
items = rfm.LocalTable(df_items, name="items").infer_metadata()
returns = rfm.LocalTable(df_returns, name="returns").infer_metadata()
views = rfm.LocalTable(df_views, name="views").infer_metadata()

### Inspecting metadata

#### Users

In [ ]:
users.print_metadata()

#### Orders

In [ ]:
orders.print_metadata()

#### Items

In [ ]:
items.print_metadata()

#### Returns

In [ ]:
returns.print_metadata()

#### Views

In [ ]:
views.print_metadata()

## Create a Graph based on the Data

In [ ]:
graph = rfm.LocalGraph.from_data({
    'users': df_users,
    'orders': df_orders,
    'items': df_items,
    'returns': df_returns,
    'views': df_views,
}, infer_metadata=True)

In [ ]:
graph.visualize();

## Materialize Graph

In [ ]:
model = rfm.KumoRFM(graph)

## Query the graph

In [ ]:
def query_graph(query, indices=None):
    df = model.predict(query, indices=indices)
    display(df)
    return df

### Identify Users for churn risk prediction
- KumoRFM can predict a value for a maximum of 1000 entities, so we need to do a preliminary filitering
- Therefore, we just look at the 100 most recent returns.

In [ ]:
df = df_returns.sort_values('date', ascending=False)
target_users = df['user_id'].drop_duplicates().head(100).to_list()
print(len(target_users))

### Perform Prediction
Predict the churn probability for the selected customers in the next 30 days.

In [ ]:
query = "PREDICT COUNT(orders.*, 0, 30, days) > 0 FOR EACH users.user_id"

In [ ]:
churn_riks = query_graph(query=query, indices=target_users)

The probabilities indicate the likelihood of each user's future order activity, where a higher false probability suggests a higher risk of churn.

In [ ]:
churn_riks_sorted = churn_riks.sort_values(by='False_PROB', ascending=False)
display(churn_riks_sorted)

## Generate a Product Recommendation for the top-5 customers that are likely to churn

Generate 3 Product Recommendations for each customer.


In [ ]:
df_top5 = churn_riks_sorted.head(5)
display(df_top5)

In [ ]:
top5= df_top5['ENTITY'].to_list()

In [ ]:
query = "PREDICT LIST_DISTINCT(orders.item_id, 0, 30, days) RANK TOP 3 FOR EACH users.user_id"

In [ ]:
recommendation = query_graph(query=query, indices=top5)

In [ ]:
recommendation_with_item_details = recommendation.rename(columns={'CLASS': 'item_id'})
recommendation_with_item_details = pd.merge(recommendation_with_item_details, df_items[['item_id', 'prod_name']], on='item_id', how='left')

display(recommendation_with_item_details[['ENTITY', 'item_id', 'SCORE', 'prod_name']])